In [ ]:
import os
import json
import base64
import requests
import tkinter as tk
from tkinter import filedialog, messagebox
import threading
from pathlib import Path
import traceback
import webbrowser


class GitHubRepoCreator:
    def __init__(self):
        # Embedded API keys (supplied by user)
        self.github_token = ""
        self.api_keys = {
            "claude": "",
            "openai": ""
        }

        self.files = []
        self.setup_ui()

    # ------------------------------------------------------------------ UI setup
    def setup_ui(self):
        self.root = tk.Tk()
        self.root.title("GitHub Repository Creator")
        self.root.geometry("800x700")
        self.root.configure(bg="#1a1a1a")

        title_label = tk.Label(
            self.root,
            text="🐙 GitHub Repository Creator",
            bg="#1a1a1a",
            fg="white",
            font=("Arial", 24, "bold"),
        )
        title_label.pack(pady=20)

        details_frame = tk.LabelFrame(
            self.root,
            text="Repository Details",
            bg="#2d2d2d",
            fg="white",
            font=("Arial", 12, "bold"),
            borderwidth=2,
            relief="groove",
        )
        details_frame.pack(padx=20, pady=10, fill="x")

        details_frame.grid_columnconfigure(1, weight=1)

        tk.Label(
            details_frame,
            text="Repository Name:",
            bg="#2d2d2d",
            fg="white",
            font=("Arial", 10),
        ).grid(row=0, column=0, sticky="w", padx=10, pady=8)
        self.repo_name = tk.Entry(
            details_frame,
            width=50,
            bg="#4b5563",
            fg="white",
            insertbackground="white",
            font=("Arial", 10),
        )
        self.repo_name.grid(row=0, column=1, padx=10, pady=8, sticky="ew")

        tk.Label(
            details_frame,
            text="Description:",
            bg="#2d2d2d",
            fg="white",
            font=("Arial", 10),
        ).grid(row=1, column=0, sticky="w", padx=10, pady=8)
        self.description = tk.Entry(
            details_frame,
            width=50,
            bg="#4b5563",
            fg="white",
            insertbackground="white",
            font=("Arial", 10),
        )
        self.description.grid(row=1, column=1, padx=10, pady=8, sticky="ew")

        tk.Label(
            details_frame,
            text="AI Provider:",
            bg="#2d2d2d",
            fg="white",
            font=("Arial", 10),
        ).grid(row=2, column=0, sticky="w", padx=10, pady=8)

        combo_frame = tk.Frame(details_frame, bg="#2d2d2d")
        combo_frame.grid(row=2, column=1, padx=10, pady=8, sticky="ew")

        self.ai_provider_var = tk.StringVar(value="manual")
        self.ai_provider = tk.OptionMenu(
            combo_frame, self.ai_provider_var, "manual", "claude", "openai"
        )
        self.ai_provider.config(
            bg="#4b5563",
            fg="white",
            font=("Arial", 10),
            highlightthickness=0,
            activebackground="#5b6573",
        )
        self.ai_provider.pack(side="left", fill="x", expand=True)

        self.generate_btn = tk.Button(
            details_frame,
            text="🤖 Generate Name",
            command=self.generate_content,
            bg="#10a37f",
            fg="white",
            font=("Arial", 10, "bold"),
            cursor="hand2",
            relief="raised",
            bd=2,
        )
        self.generate_btn.grid(row=0, column=2, padx=10, pady=8)

        self.desc_btn = tk.Button(
            details_frame,
            text="📝 Update Desc Only",
            command=self.update_description_only,
            bg="#6366f1",
            fg="white",
            font=("Arial", 10, "bold"),
            cursor="hand2",
            relief="raised",
            bd=2,
        )
        self.desc_btn.grid(row=1, column=2, padx=10, pady=8)

        file_frame = tk.LabelFrame(
            self.root,
            text="Files",
            bg="#2d2d2d",
            fg="white",
            font=("Arial", 12, "bold"),
            borderwidth=2,
            relief="groove",
        )
        file_frame.pack(padx=20, pady=10, fill="both", expand=True)
        self.file_frame = file_frame

        btn_frame = tk.Frame(file_frame, bg="#2d2d2d")
        btn_frame.pack(pady=10)

        tk.Button(
            btn_frame,
            text="📁 Add Files",
            command=self.add_files,
            bg="#3b82f6",
            fg="white",
            font=("Arial", 10, "bold"),
            cursor="hand2",
            relief="raised",
            bd=2,
        ).pack(side="left", padx=5)
        tk.Button(
            btn_frame,
            text="🗑️ Remove Selected",
            command=self.remove_selected_file,
            bg="#ef4444",
            fg="white",
            font=("Arial", 10, "bold"),
            cursor="hand2",
            relief="raised",
            bd=2,
        ).pack(side="left", padx=5)
        tk.Button(
            btn_frame,
            text="🗑️ Clear All",
            command=self.clear_files,
            bg="#dc2626",
            fg="white",
            font=("Arial", 10, "bold"),
            cursor="hand2",
            relief="raised",
            bd=2,
        ).pack(side="left", padx=5)

        list_frame = tk.Frame(file_frame, bg="#2d2d2d")
        list_frame.pack(padx=10, pady=5, fill="both", expand=True)

        scrollbar = tk.Scrollbar(list_frame)
        scrollbar.pack(side="right", fill="y")

        self.file_listbox = tk.Listbox(
            list_frame,
            bg="#4b5563",
            fg="white",
            font=("Arial", 10),
            height=10,
            yscrollcommand=scrollbar.set,
            selectmode=tk.SINGLE,
        )
        self.file_listbox.pack(side="left", fill="both", expand=True)
        scrollbar.config(command=self.file_listbox.yview)

        self.status_label = tk.Label(
            self.root,
            text="Ready to create repository",
            bg="#1a1a1a",
            fg="#10a37f",
            font=("Arial", 11),
        )
        self.status_label.pack(pady=10)

        self.create_btn = tk.Button(
            self.root,
            text="🚀 Create GitHub Repository",
            command=self.create_repository,
            bg="#059669",
            fg="white",
            font=("Arial", 14, "bold"),
            height=2,
            width=30,
            cursor="hand2",
            relief="raised",
            bd=3,
        )
        self.create_btn.pack(pady=20)

    # ------------------------------------------------------------------ helpers
    def update_status(self, message, color="#10a37f"):
        self.root.after(0, lambda: self.status_label.config(text=message, fg=color))

    # ------------------------------------------------------------------ file management
    def add_files(self):
        try:
            filenames = filedialog.askopenfilenames(title="Select files")
            for filename in filenames:
                if filename not in self.files:
                    self.files.append(filename)
                    self.file_listbox.insert(tk.END, os.path.basename(filename))
            self.update_file_count()
            if self.files and not self.repo_name.get():
                self.generate_content()
        except Exception as e:
            messagebox.showerror("Error", f"Error adding files: {e}")

    def remove_selected_file(self):
        sel = self.file_listbox.curselection()
        if sel:
            idx = sel[0]
            self.file_listbox.delete(idx)
            del self.files[idx]
            self.update_file_count()

    def clear_files(self):
        self.files.clear()
        self.file_listbox.delete(0, tk.END)
        self.update_file_count()

    def update_file_count(self):
        self.file_frame.config(text=f"Files ({len(self.files)})")

    # ------------------------------------------------------------------ name generation
    def generate_content(self):
        if not self.files:
            messagebox.showwarning("No Files", "Please add files first")
            return

        provider = self.ai_provider_var.get()
        if provider == "manual":
            self.generate_manual_content()
        else:
            self.update_status("🤖 Generating with AI...", "#f59e0b")
            self.generate_btn.config(state="disabled")
            threading.Thread(target=self._generate_ai_content_thread, daemon=True).start()

    def generate_manual_content(self):
        try:
            file_names = [os.path.basename(f).lower() for f in self.files]
            text_blob = " ".join(file_names)

            project_type = "analysis"
            company_name = ""

            if "first" in text_blob and "solar" in text_blob:
                company_name = "first-solar"
            elif "salesforce" in text_blob:
                company_name = "salesforce"
                project_type = "analytics"
            elif "tesla" in text_blob:
                company_name = "tesla"
            elif "netflix" in text_blob:
                company_name = "netflix"
                project_type = "analytics"
            else:
                if "energy" in text_blob or "solar" in text_blob:
                    project_type = "solar-energy-analysis"
                elif "finance" in text_blob:
                    project_type = "financial-analysis"
                elif "customer" in text_blob:
                    project_type = "customer-analytics"
                elif "sales" in text_blob:
                    project_type = "sales-analysis"
                elif "marketing" in text_blob:
                    project_type = "marketing-analytics"

            from datetime import datetime

            date_str = datetime.now().strftime("%Y%m")
            if company_name:
                repo_name = f"{company_name}-{project_type}-{date_str}"
                desc = (
                    f"Comprehensive {company_name.replace('-', ' ')} {project_type} "
                    f"project with {len(self.files)} files"
                )
            else:
                repo_name = f"{project_type}-{date_str}"
                desc = f"Comprehensive {project_type.replace('-', ' ')} project with {len(self.files)} files"

            self.repo_name.delete(0, tk.END)
            self.repo_name.insert(0, repo_name)
            self.description.delete(0, tk.END)
            self.description.insert(0, desc)
            self.update_status("✅ Name generated!", "#10a37f")
        except Exception as e:
            self.update_status(f"❌ Error: {e}", "#ef4444")

    def _generate_ai_content_thread(self):
        try:
            file_info = []
            for fp in self.files[:5]:
                fn = os.path.basename(fp)
                file_info.append({"name": fn, "type": Path(fn).suffix[1:] or "unknown"})

            prompt = (
                "Analyze these files and create a GitHub repository name and description.\n"
                f"Files: {json.dumps(file_info, indent=2)}\n\n"
                "Return ONLY valid JSON (no other text):\n"
                '{\n  "name": "specific-descriptive-name",\n  "description": "Brief description of the project"\n}'
            )

            provider = self.ai_provider_var.get()
            api_key = self.api_keys.get(provider)
            if not api_key:
                raise Exception(f"No API key for {provider}")

            if provider == "claude":
                url = "https://api.anthropic.com/v1/messages"
                headers = {
                    "x-api-key": api_key,
                    "anthropic-version": "2023-06-01",
                    "content-type": "application/json",
                }
                payload = {
                    "model": "claude-3-haiku-20240307",
                    "max_tokens": 300,
                    "messages": [{"role": "user", "content": prompt}],
                }
                resp = requests.post(url, headers=headers, json=payload, timeout=30)
                if resp.status_code == 200:
                    content = resp.json()["content"][0]["text"]
                    import re

                    m = re.search(r"\{.*\}", content, re.DOTALL)
                    ai_resp = json.loads(m.group()) if m else {}
                else:
                    raise Exception(f"Claude API error {resp.status_code}")
            else:
                url = "https://api.openai.com/v1/chat/completions"
                headers = {"Authorization": f"Bearer {api_key}", "Content-Type": "application/json"}
                payload = {
                    "model": "gpt-3.5-turbo",
                    "messages": [
                        {"role": "system", "content": "Return only valid JSON."},
                        {"role": "user", "content": prompt},
                    ],
                    "temperature": 0.3,
                    "max_tokens": 200,
                }
                resp = requests.post(url, headers=headers, json=payload, timeout=30)
                if resp.status_code == 200:
                    content = resp.json()["choices"][0]["message"]["content"]
                    ai_resp = json.loads(content)
                else:
                    raise Exception(f"OpenAI API error {resp.status_code}")

            def ui_update():
                self.repo_name.delete(0, tk.END)
                self.repo_name.insert(0, ai_resp.get("name", "my-project"))
                self.description.delete(0, tk.END)
                self.description.insert(0, ai_resp.get("description", "A data analysis project"))
                self.update_status("✅ AI generation complete!", "#10a37f")
                self.generate_btn.config(state="normal")

            self.root.after(0, ui_update)

        except Exception as err:
            self.update_status("❌ AI error, using manual mode", "#ef4444")
            self.generate_btn.config(state="normal")
            self.generate_manual_content()

    # ------------------------------------------------------------------ README creation
    def generate_readme(self):
        readme = f"# {self.repo_name.get()}\n\n{self.description.get()}\n\n"
        code_files = sum(Path(f).suffix in [".py", ".js", ".java", ".cpp", ".r", ".sql"] for f in self.files)
        notebooks = sum(Path(f).suffix == ".ipynb" for f in self.files)
        data_files = sum(Path(f).suffix in [".csv", ".xlsx", ".json", ".parquet"] for f in self.files)

        readme += "## 🎯 Key Achievements\n\n"
        if notebooks:
            readme += f"- **Analysis Notebooks**: {notebooks} Jupyter notebooks for data exploration\n"
        if data_files:
            readme += f"- **Data Processing**: {data_files} datasets analyzed\n"
        if code_files:
            readme += f"- **Automation**: {code_files} scripts for reproducible analysis\n"
        readme += "\n## 🛠️ Technical Stack\n\n"
        if any(f.endswith(".py") for f in self.files):
            readme += "- **Python** for data analysis\n"
        if any(f.endswith(".ipynb") for f in self.files):
            readme += "- **Jupyter** for interactive analysis\n"
        readme += "\n## 📈 Results\n\nThis project demonstrates comprehensive data analysis with actionable insights.\n\n---\n\n"
        readme += "**Author**: Scott Campbell\n"
        readme += "**Contact**: scott@cognitiqsolutions.com\n"
        readme += "**LinkedIn**: [linkedin.com/in/scott-c-52465b36b](https://linkedin.com/in/scott-c-52465b36b)\n"
        return readme

    # ------------------------------------------------------------------ GitHub upload
    def create_repository(self):
        if not self.repo_name.get() or not self.files:
            messagebox.showwarning("Missing Info", "Please provide repository name and add files")
            return
        self.update_status("🚀 Creating repository...", "#f59e0b")
        self.create_btn.config(state="disabled")
        threading.Thread(target=self._create_repository_thread, daemon=True).start()

    def _create_repository_thread(self):
        try:
            repo_resp = requests.post(
                "https://api.github.com/user/repos",
                headers={
                    "Authorization": f"token {self.github_token}",
                    "Content-Type": "application/json",
                    "Accept": "application/vnd.github.v3+json",
                },
                json={
                    "name": self.repo_name.get(),
                    "description": self.description.get(),
                    "private": False,
                },
                timeout=30,
            )
            if repo_resp.status_code != 201:
                detail = repo_resp.json().get("message", "Unknown error")
                raise Exception(f"GitHub API error: {detail}")

            repo = repo_resp.json()
            owner, name = repo["owner"]["login"], repo["name"]
            repo_url = repo["html_url"]

            self.update_status("Creating README...", "#f59e0b")
            readme_content = self.generate_readme()
            requests.put(
                f"https://api.github.com/repos/{owner}/{name}/contents/README.md",
                headers={"Authorization": f"token {self.github_token}", "Content-Type": "application/json"},
                json={
                    "message": "Initial commit: Add README",
                    "content": base64.b64encode(readme_content.encode()).decode(),
                },
                timeout=30,
            )

            total = len(self.files)
            for idx, fp in enumerate(self.files, start=1):
                self.update_status(f"Uploading file {idx}/{total}...", "#f59e0b")
                with open(fp, "rb") as fh:
                    content = base64.b64encode(fh.read()).decode()
                filename = os.path.basename(fp)
                requests.put(
                    f"https://api.github.com/repos/{owner}/{name}/contents/{filename}",
                    headers={"Authorization": f"token {self.github_token}", "Content-Type": "application/json"},
                    json={"message": f"Add {filename}", "content": content},
                    timeout=30,
                )

            self.update_status(f"✅ Repository created. Opening {repo_url}", "#10a37f")
            webbrowser.open(repo_url)
        except Exception as err:
            self.update_status(f"❌ Error: {err}", "#ef4444")
        finally:
            self.create_btn.config(state="normal")

    # ------------------------------------------------------------------ description update
    def update_description_only(self):
        if not self.files:
            messagebox.showwarning("No Files", "Please add files first")
            return
        provider = self.ai_provider_var.get()
        if provider == "manual":
            self.generate_manual_description()
        else:
            self.update_status("📝 Analyzing all files for description...", "#f59e0b")
            self.desc_btn.config(state="disabled")
            threading.Thread(target=self._generate_description_thread, daemon=True).start()

    def generate_manual_description(self):
        try:
            file_types = {}
            total_size = 0
            for fp in self.files:
                ext = Path(fp).suffix[1:].lower()
                file_types[ext] = file_types.get(ext, 0) + 1
                total_size += os.path.getsize(fp)

            notebooks = file_types.get("ipynb", 0)
            data_files = sum(file_types.get(ext, 0) for ext in ["csv", "xlsx", "xls", "json", "parquet"])
            scripts = sum(file_types.get(ext, 0) for ext in ["py", "r", "sql", "js"])

            parts = []
            if notebooks:
                parts.append(f"{notebooks} Jupyter notebook{'s' if notebooks > 1 else ''}")
            if data_files:
                parts.append(f"{data_files} data file{'s' if data_files > 1 else ''}")
            if scripts:
                parts.append(f"{scripts} script{'s' if scripts > 1 else ''}")

            desc = (
                f"Comprehensive analysis with {', '.join(parts)} "
                f"({len(self.files)} files, {total_size/1024/1024:.1f}MB)"
                if parts
                else f"Comprehensive project with {len(self.files)} files ({total_size/1024/1024:.1f}MB)"
            )

            self.description.delete(0, tk.END)
            self.description.insert(0, desc[:150])
            self.update_status("✅ Description updated!", "#10a37f")
        except Exception as e:
            self.update_status(f"❌ Error: {e}", "#ef4444")

    def _generate_description_thread(self):
        try:
            current_name = self.repo_name.get()
            file_analysis = []
            total_size = 0
            file_types_count = {}
            extracted_metrics = []

            for fp in self.files:
                fn = os.path.basename(fp)
                ext = Path(fn).suffix[1:].lower()
                total_size += os.path.getsize(fp)
                file_types_count[ext] = file_types_count.get(ext, 0) + 1

                if ext in ["py", "r", "sql", "js", "ipynb", "csv", "txt", "md", "html"]:
                    try:
                        with open(fp, "r", encoding="utf-8", errors="ignore") as f:
                            txt = f.read(50000)
                            import re

                            percents = re.findall(r"(\d+\.?\d*%)\s+([a-zA-Z\s]{3,30})", txt)
                            dollars = re.findall(r"(\$[\d,]+\.?\d*)\s+([a-zA-Z\s]{3,30})", txt)
                            improvements = re.findall(
                                r"(\d+\.?\d*\s*\w+)\s*(?:→|->|to)\s*(\d+\.?\d*\s*\w+)", txt
                            )
                            for p in percents[:5]:
                                extracted_metrics.append(f"{p[0]} {p[1].strip()}")
                            for d in dollars[:5]:
                                extracted_metrics.append(f"{d[0]} {d[1].strip()}")
                            for im in improvements[:3]:
                                extracted_metrics.append(f"{im[0]} → {im[1]}")

                            if "ETL" in txt or "pipeline" in txt:
                                file_analysis.append("ETL pipeline")
                            if "forecast" in txt.lower():
                                file_analysis.append("forecasting")
                            if "machine learning" in txt.lower() or "ML" in txt:
                                file_analysis.append("machine learning")
                            if "dashboard" in txt.lower():
                                file_analysis.append("dashboard")
                            if "real-time" in txt.lower() or "realtime" in txt.lower():
                                file_analysis.append("real time")
                    except Exception:
                        pass

            summary = (
                f"Project Analysis:\n- Repository: {current_name}\n- Total Files: {len(self.files)}\n"
                f"- Key Components: {', '.join(set(file_analysis[:5]))}\n"
                f"- Extracted Metrics: {extracted_metrics[:10]}"
            )
            prompt = (
                "Create a HIGH IMPACT description for this project that highlights BUSINESS VALUE and RESULTS.\n\n"
                f"{summary}\n\n"
                "Requirements:\n"
                "1. Start with 'Production-ready' or 'Comprehensive'\n"
                "2. Include SPECIFIC METRICS found in the files (percentages, dollar amounts, improvements)\n"
                "3. Focus on BUSINESS IMPACT not file counts\n"
                "4. Use bold for key metrics: **17.8% error reduction**, **$394,200 savings**\n"
                "5. Maximum 150 characters\n\n"
                'Return ONLY this JSON:\n{"description": "..."}'
            )

            provider = self.ai_provider_var.get()
            api_key = self.api_keys.get(provider)
            if provider == "claude":
                url = "https://api.anthropic.com/v1/messages"
                headers = {
                    "x-api-key": api_key,
                    "anthropic-version": "2023-06-01",
                    "content-type": "application/json",
                }
                payload = {
                    "model": "claude-3-haiku-20240307",
                    "max_tokens": 200,
                    "messages": [{"role": "user", "content": prompt}],
                }
                resp = requests.post(url, headers=headers, json=payload, timeout=30)
                if resp.status_code == 200:
                    import re

                    content = resp.json()["content"][0]["text"]
                    m = re.search(r"\{[^{}]*\"description\"[^{}]*\}", content, re.DOTALL)
                    ai_resp = json.loads(m.group()) if m else {}
                else:
                    raise Exception(f"Claude API error {resp.status_code}")
            else:
                url = "https://api.openai.com/v1/chat/completions"
                headers = {"Authorization": f"Bearer {api_key}", "Content-Type": "application/json"}
                payload = {
                    "model": "gpt-3.5-turbo",
                    "messages": [
                        {"role": "system", "content": "Return only valid JSON."},
                        {"role": "user", "content": prompt},
                    ],
                    "temperature": 0.3,
                    "max_tokens": 200,
                }
                resp = requests.post(url, headers=headers, json=payload, timeout=30)
                if resp.status_code == 200:
                    ai_resp = json.loads(resp.json()["choices"][0]["message"]["content"])
                else:
                    raise Exception(f"OpenAI API error {resp.status_code}")

            def update_ui():
                desc = ai_resp.get("description", "")
                # Fallback if AI did not include a metric
                if not any(sym in desc for sym in ["%", "$", "→"]) and extracted_metrics:
                    metrics_text = " and ".join(extracted_metrics[:2])
                    desc = f"Production-ready {current_name.replace('-', ' ')} achieving {metrics_text}"
                self.description.delete(0, tk.END)
                self.description.insert(0, desc[:150])
                self.update_status("✅ Description updated with business impact metrics!", "#10a37f")
                self.desc_btn.config(state="normal")

            self.root.after(0, update_ui)

        except Exception as err:
            self.update_status(f"❌ Error: {err}. Using manual description.", "#ef4444")
            self.desc_btn.config(state="normal")
            self.generate_manual_description()

    # ------------------------------------------------------------------ app launcher
    def run(self):
        self.root.mainloop()


# ---------------------------------------------------------------------- main
if __name__ == "__main__":
    try:
        app = GitHubRepoCreator()
        app.run()
    except Exception as e:
        print(f"Error: {e}")
        traceback.print_exc()
        input("Press Enter to exit...")
